
# 01 — Leer y explorar tablas **Apache Iceberg** (Spark Connect)

Este notebook te guía paso a paso para:
1) Conectarte a tu **Spark Connect Server**.  
2) Explorar el catálogo/warehouse de **Iceberg**.  
3) **Leer** una tabla (`orders_iceberg`) ya creada por tu otro job.  
4) Hacer un **perfil rápido** y, si es posible, mirar **snapshots** y **time travel**.

> Requisitos previos: tu Spark Connect Server debe tener cargado `iceberg-spark-runtime-4.0_2.13` y montar el warehouse donde escribe tu otro job, por ejemplo:  
> `/opt/tables_apache_iceberg/warehouse` (mapeado desde `demo_projects/01_CDC_GCP/tables_apache_iceberg`).


In [1]:

# === Parámetros a tu medida ===
CONNECT_URL = "sc://localhost:15002"  # URL de Spark Connect Server

# Catálogo del warehouse del job de escritura (configurado en el servidor)
ICEBERG_CATALOG = "lh"                 # catálogo configurado en el server
ICEBERG_WAREHOUSE = "/opt/tables_apache_iceberg/warehouse"  # ruta dentro del contenedor del server

# Identificadores de la tabla final
DB = "erp"
TABLE = "orders_iceberg"

# (Alternativa) Lectura por PATH directo (si no usas catálogo)
TABLE_PATH = f"{ICEBERG_WAREHOUSE}/{DB}/{TABLE}"

print("CONNECT_URL        :", CONNECT_URL)
print("ICEBERG_CATALOG    :", ICEBERG_CATALOG)
print("ICEBERG_WAREHOUSE  :", ICEBERG_WAREHOUSE)
print("TABLE FQN (catalog):", f"{ICEBERG_CATALOG}.{DB}.{TABLE}")
print("TABLE PATH         :", TABLE_PATH)


CONNECT_URL        : sc://localhost:15002
ICEBERG_CATALOG    : lh
ICEBERG_WAREHOUSE  : /opt/tables_apache_iceberg/warehouse
TABLE FQN (catalog): lh.erp.orders_iceberg
TABLE PATH         : /opt/tables_apache_iceberg/warehouse/erp/orders_iceberg


In [2]:

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .remote(CONNECT_URL)
    # Estas configs suelen estar en el servidor; aquí son redundantes pero no estorban.
    .config(f"spark.sql.catalog.{ICEBERG_CATALOG}", "org.apache.iceberg.spark.SparkCatalog")
    .config(f"spark.sql.catalog.{ICEBERG_CATALOG}.type", "hadoop")
    .config(f"spark.sql.catalog.{ICEBERG_CATALOG}.warehouse", ICEBERG_WAREHOUSE)
    .getOrCreate()
)

print("Spark version:", spark.version)


Spark version: 4.0.0


In [3]:

# Intentamos listar namespaces y tablas del catálogo de Iceberg
try:
    print(f"Namespaces en catálogo '{ICEBERG_CATALOG}':")
    spark.sql(f"SHOW NAMESPACES IN {ICEBERG_CATALOG}").show(truncate=False)
    print(f"Tablas en {ICEBERG_CATALOG}.{DB}:")
    spark.sql(f"SHOW TABLES IN {ICEBERG_CATALOG}.{DB}").show(truncate=False)
except Exception as e:
    print("No se pudo listar namespaces/tablas vía catálogo:", e)
    print("Puedes seguir leyendo por PATH si el catálogo no está disponible.")


Namespaces en catálogo 'lh':
+---------+
|namespace|
+---------+
|erp      |
+---------+

Tablas en lh.erp:
+---------+--------------+-----------+
|namespace|tableName     |isTemporary|
+---------+--------------+-----------+
|erp      |orders_iceberg|false      |
+---------+--------------+-----------+



In [4]:

from pyspark.sql import DataFrame

def load_iceberg_table(spark) -> DataFrame:
    """Intenta cargar la tabla usando el catálogo; si falla, prueba por PATH."""
    fqn = f"{ICEBERG_CATALOG}.{DB}.{TABLE}"
    # 1) Probar catálogo
    try:
        print(f"Intentando leer por catálogo: {fqn}")
        df = spark.table(fqn)
        # Forzamos una acción ligera para validar
        df.limit(1).collect()
        print("✓ Leído por catálogo.")
        return df
    except Exception as e:
        print("No se pudo leer por catálogo:", e)

    # 2) Probar por PATH
    try:
        print(f"Intentando leer por PATH: {TABLE_PATH}")
        df = spark.read.format("iceberg").load(TABLE_PATH)
        df.limit(1).collect()
        print("✓ Leído por PATH.")
        return df
    except Exception as e:
        print("No se pudo leer por PATH:", e)
        raise RuntimeError("No se pudo cargar la tabla por catálogo ni por PATH. Revisa mounts/config.") from e

df_ice = load_iceberg_table(spark)


Intentando leer por catálogo: lh.erp.orders_iceberg
✓ Leído por catálogo.


In [5]:

df_ice.printSchema()

try:
    cnt = df_ice.count()
    print("Total de filas en la tabla:", cnt)
except Exception as e:
    print("No se pudo contar filas (posible tabla muy grande). Error:", e)

df_ice.show(10, truncate=False)


root
 |-- order_id: long (nullable = true)
 |-- customer_id: long (nullable = true)
 |-- amount: decimal(12,2) (nullable = true)
 |-- status: string (nullable = true)
 |-- ts: timestamp (nullable = true)

Total de filas en la tabla: 4
+--------+-----------+------+-------+-------------------+
|order_id|customer_id|amount|status |ts                 |
+--------+-----------+------+-------+-------------------+
|1       |1          |120.50|CREATED|2025-09-28 21:18:12|
|2       |1          |89.90 |PAID   |2025-09-28 21:18:12|
|3       |2          |45.00 |CREATED|2025-09-28 21:18:12|
|4       |3          |320.00|SHIPPED|2025-09-28 21:18:12|
+--------+-----------+------+-------+-------------------+



In [ ]:

from pyspark.sql import functions as F

cols = df_ice.columns
print("Columnas:", cols)

# Conteos por status/currency si existen
if "status" in cols:
    print("\nConteo por status:")
    df_ice.groupBy("status").count().orderBy(F.desc("count")).show(truncate=False)

if "currency" in cols:
    print("\nConteo por currency:")
    df_ice.groupBy("currency").count().orderBy(F.desc("count")).show(truncate=False)

# Estadísticos de 'amount' si existe
if "amount" in cols:
    print("\nEstadísticos de 'amount':")
    df_ice.select("amount").summary().show(truncate=False)

# Nulos por columna (rápido en muestras grandes: calcula sobre todo el DF, cuidado con costos)
print("\nNulos por columna (0 = sin nulos):")
nulls = [F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in cols]
df_ice.agg(*nulls).show(truncate=False)


In [ ]:

from pyspark.sql import functions as F

fqn = f"{ICEBERG_CATALOG}.{DB}.{TABLE}"
def try_show(sql):
    try:
        spark.sql(sql).show(truncate=False)
        return True
    except Exception as e:
        print(f"No se pudo ejecutar: {sql}\n ->", e)
        return False

print(f"Intentando metadatos ACID en: {fqn}")
_ = try_show(f"SELECT * FROM {fqn}.snapshots ORDER BY committed_at DESC")
_ = try_show(f"SELECT * FROM {fqn}.history ORDER BY made_current_at DESC")


In [ ]:

# Intentamos obtener el último snapshot_id del catálogo y leer esa versión explícitamente
try:
    snaps = spark.sql(f"SELECT snapshot_id FROM {ICEBERG_CATALOG}.{DB}.{TABLE}.snapshots ORDER BY committed_at DESC")
    snap = snaps.limit(1).collect()[0]["snapshot_id"]
    print("Último snapshot_id:", snap)

    df_tt = (
        spark.read.format("iceberg")
        .option("snapshot-id", str(snap))
        .load(f"{ICEBERG_CATALOG}.{DB}.{TABLE}")
    )
    print("Time travel (10 filas):")
    df_tt.show(10, truncate=False)
except Exception as e:
    print("No fue posible realizar time travel por catálogo. Intentando por PATH...")
    try:
        # Si sabemos un snapshot_id manualmente, también sirve con PATH
        # df_tt = (spark.read.format("iceberg").option("snapshot-id", "<ID>").load(TABLE_PATH))
        # Aquí solo dejamos la mecánica impresa.
        print("Para usar time travel por PATH, necesitas un snapshot_id conocido.")
    except Exception as e2:
        print("No se pudo realizar time travel:", e2)


In [ ]:

from pyspark.sql import functions as F

# Top clientes por monto
if set(["customer_id","amount"]).issubset(df_ice.columns):
    print("Top 10 clientes por monto:")
    (df_ice.groupBy("customer_id")
          .agg(F.count("*").alias("orders"), F.sum("amount").alias("total_amount"))
          .orderBy(F.desc("total_amount"))
          .show(10, truncate=False))

# Actividad diaria por order_ts
if "order_ts" in df_ice.columns:
    print("\nActividad diaria (conteo y monto):")
    daily = (df_ice
             .withColumn("order_date", F.to_date("order_ts"))
             .groupBy("order_date")
             .agg(F.count("*").alias("orders"), F.sum("amount").alias("amount_sum"))
             .orderBy("order_date"))
    daily.show(20, truncate=False)
